<a href="https://colab.research.google.com/github/RFLORESCARTES/RFLORESCARTES/blob/main/oferta_empleo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import sys
import subprocess

def install_if_missing(packages):
    """Instala silenciosamente los paquetes que no estén disponibles."""
    for pkg in packages:
        try:
            __import__(pkg)
            print(f"✅ Paquete '{pkg}' ya instalado")
        except ImportError:
            print(f"📦 Instalando '{pkg}'...")
            try:
                subprocess.check_call([sys.executable, "-m", "pip", "install", pkg, "-q"])
                print(f"✅ '{pkg}' instalado correctamente")
            except subprocess.CalledProcessError:
                print(f"❌ Error instalando '{pkg}'. Intenta instalarlo manualmente.")

# Lista de paquetes necesarios
required_packages = [
    "schedule", "beautifulsoup4", "requests", "pandas", "lxml"
]

install_if_missing(required_packages)


📦 Instalando 'schedule'...
✅ 'schedule' instalado correctamente
📦 Instalando 'beautifulsoup4'...
✅ 'beautifulsoup4' instalado correctamente
✅ Paquete 'requests' ya instalado
✅ Paquete 'pandas' ya instalado
✅ Paquete 'lxml' ya instalado


In [15]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import json
import time
import random
from datetime import datetime, timedelta
import schedule
import threading
import re
import logging
import os
from urllib.parse import urljoin, urlparse
import warnings
import html

warnings.filterwarnings('ignore')

# ===== CONFIGURACIÓN INICIAL =====
CONFIG = {
    "email_destinatario": "raulflorescartes@gmail.com",
    "email_remitente": "",  # Se configurará desde variables de entorno
    "password_email": "",   # Se configurará desde variables de entorno
    "hora_busqueda": "07:00",
    "delay_entre_busquedas": 1,  # Nuevo: delay configurable
    "regiones_preferidas": ["RM", "V", "VIII", "Valparaíso", "Santiago", "Concepción", "Chillán", "Ñuble"],
    "palabras_clave_incluir": [
        "epidemiólogo", "epidemiología", "salud pública", "magíster",
        "vigilancia epidemiológica", "investigador salud", "coordinador vigilancia",
        "analista epidemiológico", "bioestadística", "salud poblacional"
    ],
    "palabras_clave_excluir": [
        "enfermero asistencial", "técnico", "secretaria", "vendedor",
        "práctica profesional", "freelance", "part time", "temporal"
    ],
    "requisitos_academicos": [
        "magíster", "master", "doctorado", "phd", "especialización",
        "postgrado", "posgrado"
    ]
}

# Configurar logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# ===== INSTALACIÓN Y CONFIGURACIÓN INICIAL =====
def install_dependencies():
    """Instala dependencias necesarias"""
    try:
        import schedule
        import requests
        from bs4 import BeautifulSoup
        print("✅ Todas las dependencias están instaladas")
    except ImportError:
        print("📦 Instalando dependencias...")
        os.system("pip install schedule beautifulsoup4 requests pandas lxml")
        print("✅ Dependencias instaladas correctamente")

def setup_email_credentials():
    """Configura credenciales desde input seguro"""
    from getpass import getpass
    if not CONFIG["email_remitente"]:
        print("🔐 Configuración de Email")
        CONFIG["email_remitente"] = input("Email remitente (Gmail recomendado): ").strip()
        CONFIG["password_email"] = getpass("App Password o contraseña: ")
        print("✅ Credenciales configuradas correctamente")

# ===== CLASES PRINCIPALES =====
class JobSearcher:
    """Búsqueda de empleos"""
    def __init__(self):
        self.session = requests.Session()
        self.session.headers.update({
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'
        })
        self.results = []

    def search_empleos_publicos(self):
        logger.info("🔍 Buscando en Empleos Públicos Chile...")
        jobs = []
        try:
            sample_jobs = [
                {
                    "titulo": "Epidemiólogo/a - SEREMI Salud RM",
                    "institucion": "Ministerio de Salud",
                    "descripcion": "Desarrollo de actividades de vigilancia epidemiológica y análisis de brotes",
                    "requisitos": ["Título profesional del área salud", "Magíster en Salud Pública o Epidemiología"],
                    "ubicacion": "Santiago, RM",
                    "salario": "Según escala del sector público",
                    "fecha_publicacion": datetime.now().strftime("%Y-%m-%d"),
                    "fecha_cierre": (datetime.now() + timedelta(days=15)).strftime("%Y-%m-%d"),
                    "url": "https://www.empleospublicos.cl/pub/convocatorias/avisotrabajoficha.aspx?i=123456",
                    "fuente": "empleospublicos.cl"
                }
            ]
            jobs.extend(sample_jobs)
            logger.info(f"✅ {len(sample_jobs)} empleos simulados en Empleos Públicos")
        except Exception as e:
            logger.exception("❌ Error en Empleos Públicos")
        return jobs

    def calculate_relevance_score(self, job):
        """Score normalizado"""
        score = 0
        title = job.get('titulo', '').lower()
        description = job.get('descripcion', '').lower()
        requirements = ' '.join(job.get('requisitos', [])).lower()

        # Puntaje máximo por sección
        max_title, max_req, max_desc = 40, 35, 25

        score_title = any(k in title for k in ['epidemiólogo', 'epidemiología', 'salud pública', 'vigilancia'])
        score_requirements = any(k in requirements for k in ['magíster', 'master', 'doctorado', 'phd', 'especialización'])
        score_description = sum(1 for k in ['epidemiológico', 'bioestadística', 'investigación', 'análisis'] if k in description)

        if score_title: score += max_title
        if score_requirements: score += max_req
        if score_description: score += min(max_desc, (score_description * (max_desc / 4)))

        prestigious = ['universidad de chile', 'minsal', 'isp', 'uc']
        if any(p in job.get('institucion', '').lower() for p in prestigious):
            score += 10

        return min(int(score), 100)

    def filter_jobs(self, jobs):
        """Filtrado robusto"""
        filtered = []
        for job in jobs:
            content = f"{job.get('titulo','')} {job.get('descripcion','')} {' '.join(job.get('requisitos',[]))}".lower()
            include_match = any(k in content for k in CONFIG['palabras_clave_incluir'])
            exclude_match = any(k in content for k in CONFIG['palabras_clave_excluir'])
            academic_match = any(k in content for k in CONFIG['requisitos_academicos'])
            if include_match and not exclude_match and academic_match:
                job['score'] = self.calculate_relevance_score(job)
                if job['score'] >= 70:
                    filtered.append(job)
        filtered.sort(key=lambda x: x['score'], reverse=True)
        return filtered

    def perform_search(self):
        logger.info("🚀 Ejecutando búsqueda completa...")
        all_jobs = []
        all_jobs.extend(self.search_empleos_publicos())
        time.sleep(CONFIG['delay_entre_busquedas'])
        filtered_jobs = self.filter_jobs(all_jobs)
        logger.info(f"📊 Búsqueda: {len(all_jobs)} encontradas, {len(filtered_jobs)} relevantes")
        self.results = filtered_jobs
        return filtered_jobs

class EmailReporter:
    """Envía reportes por email"""
    def __init__(self):
        self.smtp_server = "smtp.gmail.com"
        self.smtp_port = 587

    def generate_html_report(self, jobs, search_date):
        """Reporte HTML seguro"""
        total_jobs = len(jobs)
        avg_score = sum(job['score'] for job in jobs) / total_jobs if total_jobs > 0 else 0
        html_template = f"""<html><body><h1>🔬 Reporte - {search_date}</h1>"""
        html_template += f"<p>Ofertas relevantes: {total_jobs} | Score promedio: {avg_score:.1f}%</p>"
        if total_jobs == 0:
            html_template += "<p>No se encontraron ofertas relevantes hoy.</p>"
        else:
            for job in jobs:
                titulo = html.escape(job.get('titulo', ''))
                descripcion = html.escape(job.get('descripcion', ''))
                institucion = html.escape(job.get('institucion', ''))
                html_template += f"""
                <hr>
                <h3>{titulo} ({job['score']}%)</h3>
                <p><b>Institución:</b> {institucion}</p>
                <p>{descripcion}</p>
                <p><a href="{job.get('url','')}" target="_blank">Ver oferta completa</a></p>
                """
        html_template += "</body></html>"
        return html_template

    def send_report(self, jobs, search_date):
        """Envío robusto con logging de error completo"""
        try:
            msg = MIMEMultipart('alternative')
            msg['From'] = CONFIG['email_remitente']
            msg['To'] = CONFIG['email_destinatario']
            msg['Subject'] = f"🔬 Ofertas Salud Pública - {search_date} - {len(jobs)} encontradas"
            html_part = MIMEText(self.generate_html_report(jobs, search_date), 'html', 'utf-8')
            msg.attach(html_part)
            server = smtplib.SMTP(self.smtp_server, self.smtp_port)
            server.starttls()
            server.login(CONFIG['email_remitente'], CONFIG['password_email'])
            server.sendmail(CONFIG['email_remitente'], CONFIG['email_destinatario'], msg.as_string())
            server.quit()
            logger.info(f"✅ Reporte enviado a {CONFIG['email_destinatario']}")
            return True
        except Exception:
            logger.exception("❌ Error enviando email")
            return False

# ===== CONFIGURACIÓN PARA GOOGLE COLAB =====
def colab_quick_start():
    print("📱 MODO GOOGLE COLAB - INICIO RÁPIDO")
    from getpass import getpass
    CONFIG['email_remitente'] = input("Email remitente (Gmail): ").strip()
    CONFIG['password_email'] = getpass("App Password: ")
    CONFIG['email_destinatario'] = input("Email destinatario (receptor del reporte): ").strip()
    searcher = JobSearcher()
    reporter = EmailReporter()
    jobs = searcher.perform_search()
    search_date = datetime.now().strftime("%d/%m/%Y")
    reporter.send_report(jobs, search_date)



In [16]:
# ===== COLAB QUICK START: INSTALACIÓN + CONFIGURACIÓN + BÚSQUEDA DE PRUEBA =====
def colab_quick_start():
    """Inicio rápido con instalación de dependencias y configuración de email en Google Colab"""
    print("📱 MODO GOOGLE COLAB - INICIO RÁPIDO")
    print("="*40)
    import sys, subprocess

    # Instalar dependencias necesarias
    def install_if_missing(packages):
        for pkg in packages:
            try:
                __import__(pkg)
                print(f"✅ '{pkg}' ya instalado")
            except ImportError:
                print(f"📦 Instalando '{pkg}'...")
                try:
                    subprocess.check_call([sys.executable, "-m", "pip", "install", pkg, "-q"])
                    print(f"✅ '{pkg}' instalado correctamente")
                except subprocess.CalledProcessError:
                    print(f"❌ Error instalando '{pkg}'")

    required_packages = ["schedule", "beautifulsoup4", "requests", "pandas", "lxml"]
    install_if_missing(required_packages)

    # Configuración de email
    from getpass import getpass
    CONFIG['email_remitente'] = input("📧 Email remitente (Gmail): ").strip()
    CONFIG['password_email'] = getpass("🔑 App Password: ")
    CONFIG['email_destinatario'] = input("📨 Email destinatario (receptor del reporte): ").strip()

    # Crear instancia y ejecutar búsqueda de prueba
    searcher = JobSearcher()
    reporter = EmailReporter()

    print("\n🔎 Ejecutando búsqueda de prueba...")
    jobs = searcher.perform_search()
    search_date = datetime.now().strftime("%d/%m/%Y")

    reporter.send_report(jobs, search_date)
    print("✅ Búsqueda de prueba completada y reporte enviado.")


In [18]:
colab_quick_start()

📱 MODO GOOGLE COLAB - INICIO RÁPIDO
✅ 'schedule' ya instalado
📦 Instalando 'beautifulsoup4'...
✅ 'beautifulsoup4' instalado correctamente
✅ 'requests' ya instalado
✅ 'pandas' ya instalado
✅ 'lxml' ya instalado
📧 Email remitente (Gmail): raulflorescartes@gmail.com
🔑 App Password: ··········
📨 Email destinatario (receptor del reporte): raulflorescartes@gmail.com

🔎 Ejecutando búsqueda de prueba...
✅ Búsqueda de prueba completada y reporte enviado.
